# Install libraries

In [30]:
!pip install dash-bootstrap-components
!pip install dash plotly
!pip install pandas
!pip install folium dash dash-bootstrap-components

# Dashboard - existing collaborations map

In [ ]:
import os
import dash
import dash_bootstrap_components as dbc
from dash import html, dcc
from dash.dependencies import Input, Output
import folium
from folium.plugins import MarkerCluster
from folium import DivIcon
import pandas as pd
import numpy as np

# -------------------------
# Data Loading
# ------------------------

def convert_dataframe_types(df):
    """
    Convert all DataFrame columns to standard Python types to ensure JSON serialization.
    """
    for col in df.select_dtypes(include=['int64', 'float64']).columns:
        df[col] = df[col].apply(lambda x: int(x) if pd.notna(x) else None)
    return df



def load_data(participants_path='Data_clean/01_participants_with_geo.csv',
              projects_path='Data_clean/02_projects.csv'):
    participants = pd.read_csv(participants_path)
    projects = pd.read_csv(projects_path)

    projects_geo = projects.merge(
        participants[['full_name', 'Latitude', 'Longitude', 'Affiliation']],
        on='full_name',
        how='left'
    )

    # Convert all DataFrame columns to standard Python types
    participants = convert_dataframe_types(participants)
    projects_geo = convert_dataframe_types(projects_geo)

    return participants, projects_geo


participants, projects_geo = load_data()

# -------------------------
# Bézier Curve Points
# -------------------------
def generate_bezier_points(start, end, curvature=0.1, n_points=20, offset=0.01):
    """
    Generate Bézier curve points between start and end, adding small offset for identical points.
    """
    # Add an offset if points are too close
    if np.allclose(start, end, atol=0.0001):
        end = [end[0] + offset, end[1] + offset]

    # Calculate the midpoint with curvature
    midpoint = [
        (start[0] + end[0]) / 2 + curvature,  # Adjust latitude for curvature
        (start[1] + end[1]) / 2              # Longitude remains midway
    ]

    # Generate Bézier curve points
    t_values = np.linspace(0, 1, n_points)
    curve_points = [
        (
            (1 - t) ** 2 * start[0] + 2 * (1 - t) * t * midpoint[0] + t ** 2 * end[0],
            (1 - t) ** 2 * start[1] + 2 * (1 - t) * t * midpoint[1] + t ** 2 * end[1]
        )
        for t in t_values
    ]
    return curve_points


# -------------------------
# Map Helper Functions
# -------------------------
def create_base_map(location=[51, 10], zoom=5, tiles="cartodbpositron"):
    return folium.Map(location=location, zoom_start=zoom, tiles=tiles)

def generate_project_details_html(researcher_name, filtered_projects):
    """
    Generate project details HTML for the popup while removing duplicates.
    """
    researcher_projects = filtered_projects[filtered_projects['full_name'] == researcher_name]
    if researcher_projects.empty:
        return "<li>No project</li>"

    # Use a set to track unique projects
    unique_projects = set()
    project_details = ""

    for _, proj in researcher_projects.iterrows():
        project_key = (proj['Project'], proj['Project Type'])  # Key to identify duplicates
        if project_key not in unique_projects:
            unique_projects.add(project_key)
            collaborators = ', '.join(
                filtered_projects[filtered_projects['Project'] == proj['Project']]['full_name'].unique()
            )
            project_details += f"""
            <li><b>{proj['Project']}</b> ({proj['Project Type']})<br>
            <i>Collaborators:</i> {collaborators}</li>
            """

    return project_details


def create_popup_content(row, project_details):
    popup_content = f"""
    <div class="popup-content">
        <h5 class="text-center">{row['full_name']}</h5>
        <p class="text-center text-muted">{row['Affiliation']}</p>
        <div class="text-center mb-2">
            <img src="{row['Photo']}" style="width:100px; height:100px; border-radius:50%; object-fit:cover;">
        </div>
        <h6>Projects:</h6>
        <ul class="project-list">
            {project_details}
        </ul>
    </div>
    """
    return popup_content




def add_researcher_marker(row, filtered_projects, marker_cluster, highlight=False, opacity=1.0):
    """
    Add researcher marker with photos intact, without overcomplicating logic.
    """
    project_details = generate_project_details_html(row['full_name'], filtered_projects)
    popup_content = create_popup_content(row, project_details)

    # Ensure photo URL is used directly
    photo_url = row['Photo'] if pd.notna(row['Photo']) else "https://via.placeholder.com/40"
    border_style = 'border:2px solid red;' if highlight else ''
    opacity_style = f"opacity:{opacity};"

    # Custom marker HTML
    icon_html = f"""
    <div style="width:40px; height:40px; border-radius:50%; overflow:hidden; {border_style} {opacity_style}">
        <img src="{photo_url}" style="width:100%; height:100%; object-fit:cover;">
    </div>
    """
    icon = folium.DivIcon(html=icon_html, icon_size=(40, 40), icon_anchor=(20, 20))

    # Add marker
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=folium.Popup(popup_content, max_width=300),
        icon=icon,
        tooltip=row['full_name']
    ).add_to(marker_cluster)


def draw_affiliation_lines(filtered_projects, map_obj):
    """
    Draw lines between collaborators, skipping identical coordinates.
    """
    project_groups = filtered_projects.groupby("Project")
    
    # Process each project group
    for _, group in project_groups:
        group = group.dropna(subset=["Latitude", "Longitude"])
        if group.shape[0] < 2:  # Skip if fewer than 2 collaborators
            continue
        
        coords = group[['Latitude', 'Longitude']].values
        seen_coords = set()  # Track processed coordinates to avoid duplicates
        
        for i in range(len(coords)):
            for j in range(i + 1, len(coords)):
                start, end = tuple(coords[i]), tuple(coords[j])

                # Skip drawing if start and end are identical
                if start == end:
                    continue

                # Add the coordinates to the 'seen' set
                if (start, end) in seen_coords or (end, start) in seen_coords:
                    continue  # Avoid duplicate lines
                seen_coords.add((start, end))

                # Generate and draw the Bézier curve
                bezier_points = generate_bezier_points(start, end, curvature=0.1)
                line_color = "blue" if group["Project Type"].iloc[0] == "Main Project" else "green"

                folium.PolyLine(
                    locations=bezier_points,
                    color=line_color,
                    weight=2.5,
                    opacity=0.8
                ).add_to(map_obj)




def add_map_legend(m):
    legend_html = '''
    <div style="position: fixed; 
                bottom: 50px; left: 50px; z-index:9999;
                background-color:white; padding:10px; 
                border-radius:6px; box-shadow:0 2px 8px rgba(0,0,0,0.1);
                font-size:14px;">
    <b>Legend</b><br>
    <span style="color:blue;">■</span> Main Projects<br>
    <span style="color:green;">■</span> Short-Term Collaborations<br>
    </div>
    '''
    m.get_root().html.add_child(folium.Element(legend_html))


def generate_folium_map(participants, projects_geo, filter_types=["All"], selected_researcher=None):
    """
    Generate the folium map, showing collaborators for the selected researcher and avoiding duplicates.
    """
    # Local set to track markers and prevent duplicates
    seen_researchers = set()

    # Default map
    default_location = [51, 10]
    zoom_level = 5
    m = create_base_map(location=default_location, zoom=zoom_level)
    marker_cluster = MarkerCluster(maxClusterRadius=5).add_to(m)

    # Filter projects
    if "All" in filter_types:
        filtered_projects = projects_geo
    else:
        filtered_projects = projects_geo[projects_geo['Project Type'].isin(filter_types)]

    # Handle researcher filter
    collaborators = set()
    if selected_researcher:
        researcher_projects = filtered_projects[filtered_projects['full_name'] == selected_researcher]['Project'].unique()
        filtered_projects = filtered_projects[filtered_projects['Project'].isin(researcher_projects)]
        collaborators = set(filtered_projects['full_name'].unique())

    # Add markers
    for _, row in participants.iterrows():
        if (row['full_name'], row['Latitude'], row['Longitude']) not in seen_researchers:
            seen_researchers.add((row['full_name'], row['Latitude'], row['Longitude']))
            highlight = row['full_name'] in collaborators or row['full_name'] == selected_researcher
            opacity = 1.0 if highlight else 0.3
            add_researcher_marker(row, filtered_projects, marker_cluster, highlight=highlight, opacity=opacity)

    # Draw lines for collaborations
    draw_affiliation_lines(filtered_projects, m)

    # Add legend and save
    add_map_legend(m)
    map_path = "map.html"
    m.save(map_path)
    return map_path




# -------------------------
# Dash App
# -------------------------
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.DARKLY])

app.index_string = """
<!DOCTYPE html>
<html>
<head>
    <title>ViCom CollabMap</title>
    <link href="https://fonts.googleapis.com/css2?family=Open+Sans:wght@400;600&display=swap" rel="stylesheet">
    <link rel="stylesheet" href="/assets/style.css">
    {%favicon%}
    {%css%}
</head>
<body>
    {%app_entry%}
    {%config%}
    {%scripts%}
    {%renderer%}
</body>
</html>
"""

app.layout = dbc.Container(
    fluid=True,
    children=[
        dbc.Row([
            dbc.Col(html.H1("ViCom CollabMap", className="main-title"), width=12)
        ]),
        dbc.Row([
            dbc.Col([
                dbc.Card([
                    dbc.CardHeader("Search & Filters"),
                    dbc.CardBody([
                        html.Label("Search for Researcher:", className="filter-label"),
                        dcc.Dropdown(
                            id='researcher-dropdown',
                            options=[{'label': name, 'value': name} for name in participants['full_name'].unique()],
                            placeholder="Select a researcher",
                            className="mb-3"
                        ),
                        html.Label("Collaboration Type:", className="filter-label"),
                        dbc.Checklist(
                            options=[
                                {"label": "Main Projects", "value": "Main Project"},
                                {"label": "Short-Term Collaborations", "value": "Short-Term Collaboration"}
                            ],
                            value=["Main Project", "Short-Term Collaboration"],
                            id="collab-checkboxes"
                        )
                    ])
                ])
            ], width=3),
            dbc.Col([
                dbc.Card([
                    dbc.CardBody([
                        html.Iframe(
                            id="map-frame",
                            srcDoc=open("map.html", "r").read() if os.path.exists("map.html") else "",
                            className="map-frame"
                        )
                    ])
                ])
            ], width=9)
        ])
    ]
)

@app.callback(
    Output('map-frame', 'srcDoc'),
    [Input('researcher-dropdown', 'value'),
     Input('collab-checkboxes', 'value')]
)
def update_map(selected_researcher, collab_types):
    print("Selected Researcher:", selected_researcher)
    print("Selected Collaboration Types:", collab_types)

    filter_types = collab_types if collab_types else ["All"]
    print("Final Filter Types:", filter_types)

    map_path = generate_folium_map(participants, projects_geo, filter_types, selected_researcher)
    if os.path.exists(map_path):
        print("Map generated successfully:", map_path)
        return open(map_path, "r").read()
    else:
        print("Error: Map file not found!")
        return "Map file generation failed."
    
if __name__ == '__main__':
    participants, projects_geo = load_data()
    app.run_server(debug=True)




Selected Researcher: None
Selected Collaboration Types: ['Main Project', 'Short-Term Collaboration']
Final Filter Types: ['Main Project', 'Short-Term Collaboration']
Map generated successfully: map.html
Selected Researcher: None
Selected Collaboration Types: ['Main Project', 'Short-Term Collaboration']
Final Filter Types: ['Main Project', 'Short-Term Collaboration']
Map generated successfully: map.html
Selected Researcher: None
Selected Collaboration Types: ['Main Project', 'Short-Term Collaboration']
Final Filter Types: ['Main Project', 'Short-Term Collaboration']
Map generated successfully: map.html
Selected Researcher: None
Selected Collaboration Types: ['Main Project', 'Short-Term Collaboration']
Final Filter Types: ['Main Project', 'Short-Term Collaboration']
Map generated successfully: map.html
Selected Researcher: None
Selected Collaboration Types: ['Short-Term Collaboration']
Final Filter Types: ['Short-Term Collaboration']
Map generated successfully: map.html
Selected Research